## Problem:
The problem is that LWC and NBC are too low by a factor of 2.
The origin of this issue is not yet clear!

## Solution:
simply multiply the scale factor of the parameter dataset by ``2``!!

In [ ]:
import xarray as xr
from pathlib import Path
from sdm_eurec4a import RepositoryPath

RepoPaths = RepositoryPath("levante")

### Data loading

In [ ]:
OBS_data_dir = RepoPaths.data_dir
input_path = OBS_data_dir / Path("model/input_v4.0/")
assert input_path.is_dir()
output_path = OBS_data_dir / Path("model/input_v4.1/")
output_path.mkdir(exist_ok=True, parents=False)

psd_name = "particle_size_distribution_parameters.nc"
psd_linear_name = "particle_size_distribution_parameters_linear_space.nc"
potential_temperature_name = "potential_temperature_parameters.nc"
relative_humidity_name = "relative_humidity_parameters.nc"
pressure_name = "pressure_parameters.nc"
names = [psd_name, psd_linear_name, potential_temperature_name, relative_humidity_name, pressure_name]
# copy all these file to the new directory

### Copy and adjust datasets if necessary

In [ ]:
for name in names:
    ds = xr.open_dataset(input_path / Path(name))
    ds.to_netcdf(output_path / Path(name))
    ds.close()
# modify the psd file

for name in [psd_name, psd_linear_name]:
    ds_parameters = xr.open_dataset(input_path / Path(name))
    ds_parameters["scale_factor1"] = 2 * ds_parameters["scale_factor1"]
    ds_parameters["scale_factor2"] = 2 * ds_parameters["scale_factor2"]
    ds_parameters.to_netcdf(output_path / Path(name))
    ds_parameters.close()

### Validate the datasets are the same.
nans in div are due to 0 / 0 calculation.
The diff in scale factors is expected, BUT the div needs to be 2!

In [ ]:
for name in names:
    print("------")
    print(name)
    ds = xr.open_dataset(input_path / Path(name))
    ds_new = xr.open_dataset(output_path / Path(name))

    ljust_front = 20
    ljust_num = 11

    print(
        "".ljust(ljust_front),
        "div mean".ljust(ljust_num),
        "div std".ljust(ljust_num),
        "diff mean".ljust(ljust_num),
        "diff std".ljust(ljust_num),
    )
    for var in list(ds.variables):
        var = str(var)
        div_mean = f"{((ds_new[var] / ds[var]).mean().values):.2e}".ljust(ljust_num)
        div_std = f"{((ds_new[var] / ds[var]).std().values):.2e}".ljust(ljust_num)
        diff_mean = f"{((ds_new[var]  - ds[var]).mean().values):.2e}".ljust(ljust_num)
        diff_std = f"{((ds_new[var]  - ds[var]).std().values):.2e}".ljust(ljust_num)

        print(var.ljust(ljust_front), div_mean, div_std, diff_mean, diff_std)
    ds.close()
    ds_new.close()